In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandahouse as ph
import datetime
from datetime import timedelta
from operator import attrgetter
%matplotlib inline

## По каждому товару определить, в какой день недели товар чаще всего покупается.

In [ ]:
# Для дальнейших вычислений пересоздадим соединение датафреймов customers и items
customers_items = customers.merge(items)

In [ ]:
#Добавим к нашему соединенному датафрейму (customers_items) третий датафрейм. Соеденим по дефолту во избежание лишних пропусков.
all_df = customers_items.merge(orders)
all_df.shape

(112650, 18)

In [ ]:
# Проверим количество пропущенных значений

all_df.isna().sum()

# Исходя из нашего изначально принятого правила, что покупкой считается и оплата и подтвержденная доставка, данные 15 товаров
# мы учитывать не будем

customer_id                         0
customer_unique_id                  0
customer_zip_code_prefix            0
customer_city                       0
customer_state                      0
order_id                            0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_carrier_date     1194
order_delivered_customer_date    2454
order_estimated_delivery_date       0
order_item_id                       0
product_id                          0
seller_id                           0
shipping_limit_date                 0
price                               0
freight_value                       0
dtype: int64

In [ ]:
# Считаем количество уникальных проданных товаров
all_df[all_df.order_approved_at.notna()].query('order_status == "delivered"').product_id.nunique()

32213

In [ ]:
# Найдем общее количество проданных товарам за каждый день

week_day_products = all_df[all_df.order_approved_at.notna()] \
                                 .query('order_status == "delivered"') \
                                 .groupby(all_df.order_approved_at.dt.day_name()) \
                                 .agg({'product_id': 'value_counts'}) \
                                 .rename(columns={'product_id': 'product_counts'}) \
                                 .reset_index() \
                                 .rename(columns={'order_approved_at': 'week_day'})

In [ ]:
week_day_products

,week_day,product_id,product_counts
0,Friday,aca2eb7d00ea1a7b8ebd4e68314663af,92
1,Friday,422879e10f46682990de24d770e7f83d,87
2,Friday,53759a2ecddad2bb87a079a1f1519f73,68
3,Friday,99a4788cb24856965c36a24e339b6058,66
4,Friday,389d119b48cf3043d311335e499d9c6b,59
...,...,...,...
59831,Wednesday,ffe013e1b4603e3b0b02fbb159d5b400,1
59832,Wednesday,ffe0fc4e02c3559643ac063fa5cf9d07,1
59833,Wednesday,ffe75578163d45caa52bca3529cb511e,1
59834,Wednesday,ffedbd68fa6f44e788ff6c2db8094715,1


In [ ]:
# Уникальные коды товаров и их максимальное число покупок
max_products_counts = week_day_products.groupby('product_id', as_index=False).agg({'product_counts':'max'})

In [ ]:
max_products_counts

,product_id,product_counts
0,00066f42aeeb9f3007548bb9d3f33c38,1
1,00088930e925c41fd95ebfe695fd2655,1
2,0009406fd7479715e4bef61dd91f2462,1
3,000b8f95fcb9e0096488278317764d19,1
4,000d9be29b5207b54e86aa1b1ac54872,1
...,...,...
32208,fff6177642830a9a94a0f2cba5e476d1,1
32209,fff81cc3158d2725c0655ab9ba0f712c,1
32210,fff9553ac224cec9d15d49f5a263411f,1
32211,fffdb2d0ec8d6a61f0a0a0db3f25b441,3


In [ ]:
# Итоговый результат
max_week_day_products_counts = week_day_products.merge(max_products_counts)

In [ ]:
# Для более читабельного вида (и для красоты) расположим дни согласно недели 
weeks = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
max_week_day_products_counts ['week_day'] = pd.Categorical(max_week_day_products_counts['week_day'], categories=weeks, ordered=True)
max_week_day_products_counts = max_week_day_products_counts.sort_values('week_day')

In [ ]:
max_week_day_products_counts

,week_day,product_id,product_counts
14912,Monday,a272b490eba6b022384f14ecb9afc7a9,1
10749,Monday,601d52789faa7ded8408d63ccfcb4800,2
10750,Monday,6171e0ffb665e527d521c2435e065644,2
10751,Monday,627c9ddcab5043db4f8816ce2da1d31c,2
10752,Monday,6291f9025c6b863241bb80ebafa4434e,2
...,...,...,...
23055,Sunday,8a5835aeef83efa6aa947f84f92deb0e,2
23056,Sunday,8b1b115b8dc96bdfa740d4267019a201,2
23058,Sunday,8b6ede776c9602a1db79d27547ff8253,2
23043,Sunday,8374b39a15882a19ce4558f13064b55c,2
